# Web Server Logs Analysis

De forma general, un server log es un archivo de log generado por un servidor con una lista de actividades que se ejecutan. En este caso tenemos un web server log el cuál mantiene un historial de las peticiones realizadas a la página. Este tipo de server logs tienen un formato standard Common Log Format (https://en.wikipedia.org/wiki/Common_Log_Format). Es una práctica general el analizar estos logs para sacar distintas conclusiones, localizar ataques, errores comunes, etc.

En nuestro caso tenemos el dataset de los web server logs de la NASA, que están compuestos por este tipo de registros:

133.43.96.45 - - [01/Aug/1995:00:00:23 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713

Por lo que tenemos los siguientes campos:

1. **Host:** 133.43.96.45
2. **User-identifier:** En este dataset, todos los campos estarán con un "-" que significa que faltan esos datos, por lo que obviaremos este campo.
3. **Userid:** También es obviado
4. **Date:** 01/Aug/1995:00:00:23 -0400, como podemos ver está en formato dd/MMM/yyyy:HH:mm:ss y el campo final "-0400" sería el timezone que en este caso omitiremos, además haremos una transformación de los meses a forma numérica.
5. **Request Method:** GET
6. **Resource:** /images/launch-logo.gif, sería el recurso al que se accede en esta petición.
7. **Protocol:** HTTP:/1.0, y por último en esta parte entre comillas tendríamos el protocolo utilizado al ser logs de 1995, seguramente sea el único protocolo utilizado.
8. **HTTP status code:** 200, existen distintos códigos de estado de HTTP. Aquí tienes más información: https://en.wikipedia.org/wiki/List_of_HTTP_status_codes
9. **Size:** 1713. El tamaño del objeto recibido por el cliente en bytes. En casos de error del cliente, este campo no se encuentra por lo que al igual que en los userid será indicado con un "-", tenerlo en cuenta.

Ahora que ya entendemos que se encuentra dentro de nuestro web server log, vamos a pasar a analizarlo. Primero debemos cargar el archivo como un fichero de texto normal y realizar las transformaciones pertinentes. A la hora de limpiar y estructurar nuestro dataset utilizaremos expresiones regulares para recoger los campos que necesitamos.

Ahora que ya entendemos qué se encuentra dentro de nuestro web sever log, vamos a pasar a analizarlo. Primero debemos cargar el archivo como un archivo de texto normal y realizar las transformaciones pertinentes, a la hora de limpiar y estructurar nuestro dataset utilizaremos expresiones regulares para recoger los campos que necesitamos.

Guardaremos nuestro dataframe ya estructurado en formato parquet. Y este lo leeremos para realizar nuestro análisis.

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("NASA")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2111027.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1643372301519)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@61aaafae


In [2]:
// Leo el dataset como texto
val dfAug = spark.read.format("text").load("access_log_Aug95")
val dfJul = spark.read.format("text").load("access_log_Jul95")

val df= dfJul.union(dfAug)

dfAug: org.apache.spark.sql.DataFrame = [value: string]
dfJul: org.apache.spark.sql.DataFrame = [value: string]
df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [value: string]


In [3]:
df.show(numRows=5,truncate=false)

+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------

In [4]:
// Utilizo un poco de brujería regex para asignar cada valor a cada columna.
val parsed_df = df.select(regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=1).alias("Host"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=2).alias("Date"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=3).alias("RequestMethod"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=4).alias("Resource"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=5).alias("Protocol"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=6).alias("HTTPstatuscode"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" ([\d]*) (.*)""",groupIdx=7).alias("Size")).na.drop()

parsed_df.show(numRows=10, truncate=false)

+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|Host                |Date                      |RequestMethod|Resource                                       |Protocol|HTTPstatuscode|Size|
+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|199.72.81.55        |01/Jul/1995:00:00:01 -0400|GET          |/history/apollo/                               |HTTP/1.0|200           |6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00:06 -0400|GET          |/shuttle/countdown/                            |HTTP/1.0|200           |3985|
|199.120.110.21      |01/Jul/1995:00:00:09 -0400|GET          |/shuttle/missions/sts-73/mission-sts-73.html   |HTTP/1.0|200           |4085|
|burger.letters.com  |01/Jul/1995:00:00:11 -0400|GET          |/shuttle/countdown/liftoff.html                |HTTP/1.0|304           |0   |
|199.120.110.

parsed_df: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 5 more fields]


In [30]:
// Utilizo un poco de brujería regex para asignar cada valor a cada columna.
val parsed_df = df.select(regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=1).alias("Host"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=2).alias("Date"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=3).alias("RequestMethod"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=4).alias("Resource"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=5).alias("Protocol"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=6).alias("HTTPstatuscode"),
regexp_extract($"value", exp="""(.*) - - \[(.*)\] "(\S*) (\S*) *(\S*)" (.*) (.*)""",groupIdx=7).alias("Size")).na.drop()

parsed_df.show(numRows=10, truncate=false)

+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|Host                |Date                      |RequestMethod|Resource                                       |Protocol|HTTPstatuscode|Size|
+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|199.72.81.55        |01/Jul/1995:00:00:01 -0400|GET          |/history/apollo/                               |HTTP/1.0|200           |6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00:06 -0400|GET          |/shuttle/countdown/                            |HTTP/1.0|200           |3985|
|199.120.110.21      |01/Jul/1995:00:00:09 -0400|GET          |/shuttle/missions/sts-73/mission-sts-73.html   |HTTP/1.0|200           |4085|
|burger.letters.com  |01/Jul/1995:00:00:11 -0400|GET          |/shuttle/countdown/liftoff.html                |HTTP/1.0|304           |0   |
|199.120.110.

parsed_df: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 5 more fields]


In [34]:
//parsed_df.where(($"Protocol"!=="STS-69</a><p>") && ($"Protocol" !== "a")).where($"host"==="").show()
parsed_df.where($"Protocol"==="a").show()

+----------------+--------------------+-------------+--------------------+--------+--------------+----+
|            Host|                Date|RequestMethod|            Resource|Protocol|HTTPstatuscode|Size|
+----------------+--------------------+-------------+--------------------+--------+--------------+----+
|dsl.rhilinet.gov|16/Aug/1995:11:17...|          GET|/software/winvn/w...|       a|           404|   -|
+----------------+--------------------+-------------+--------------------+--------+--------------+----+



In [5]:
val parsed_limpio=parsed_df.where(($"Host" !== "") && ($"Date" !== "") && ($"RequestMethod" !== "") && ($"Resource" !== "") &&
                                  ($"Protocol" !== "") && ($"HTTPstatuscode" !== "") && ($"Size" !==""))

parsed_limpio: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Host: string, Date: string ... 5 more fields]


In [6]:
// Guardo el dataset estructurado en formato Parquet.
//parsed_limpio.write.parquet("C:/Users/rafael.gomez/Documents/EjerciciosSpark/NASA/dfparsed")
parsed_df129.188.154.200 - - [01/Jul/1995:00:03:58 -0400] "GET / HTTP/1.0" 200 7074.write.format("parquet")
                .mode("overwrite")
                .save("C:/Users/rafael.gomez/Documents/EjerciciosSpark/NASA/dfparsed")

In [7]:
// Vuelvo a leerlo porque... lo pide el enunciado.
val parqDF = spark.read.parquet("C:/Users/rafael.gomez/Documents/EjerciciosSpark/NASA/dfparsed")

parqDF.show(5,false)

+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|Host                |Date                      |RequestMethod|Resource                                       |Protocol|HTTPstatuscode|Size|
+--------------------+--------------------------+-------------+-----------------------------------------------+--------+--------------+----+
|199.72.81.55        |01/Jul/1995:00:00:01 -0400|GET          |/history/apollo/                               |HTTP/1.0|200           |6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00:06 -0400|GET          |/shuttle/countdown/                            |HTTP/1.0|200           |3985|
|199.120.110.21      |01/Jul/1995:00:00:09 -0400|GET          |/shuttle/missions/sts-73/mission-sts-73.html   |HTTP/1.0|200           |4085|
|burger.letters.com  |01/Jul/1995:00:00:11 -0400|GET          |/shuttle/countdown/liftoff.html                |HTTP/1.0|304           |0   |
|199.120.110.

parqDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 5 more fields]


### ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos

In [8]:
parqDF.select($"Protocol").distinct().show(10)

+-------------+
|     Protocol|
+-------------+
|       HTTP/*|
|            a|
|    HTTP/V1.0|
|     HTTP/1.0|
|STS-69</a><p>|
+-------------+



### ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.

In [9]:
parqDF.groupBy($"HTTPstatuscode").agg(count("*")).orderBy(desc("count(1)")).show()

+--------------+--------+
|HTTPstatuscode|count(1)|
+--------------+--------+
|           200| 3094323|
|           304|  266764|
|           302|   72966|
|           404|   20628|
|           403|     225|
|           500|      65|
|           501|      41|
+--------------+--------+



###  ¿Y los métodos de petición (verbos) más utilizados?

In [10]:
parqDF.groupBy($"RequestMethod").agg(count("*")).orderBy(desc("count(1)")).show()

+-------------+--------+
|RequestMethod|count(1)|
+-------------+--------+
|          GET| 3446875|
|         HEAD|    7915|
|         POST|     222|
+-------------+--------+



### ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

In [11]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType

parqDF.withColumn("size", col("size").cast(IntegerType)).orderBy(desc("size")).select(col("resource"),col("size")).show(1,false)

+---------------------------------------+-------+
|resource                               |size   |
+---------------------------------------+-------+
|/shuttle/countdown/video/livevideo.jpeg|6823936|
+---------------------------------------+-------+
only showing top 1 row



import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType


### Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log

In [12]:
parqDF.withColumn("Size",col("Size").cast(IntegerType)).groupBy(col("resource")).agg(count("resource").alias("count")).orderBy(desc("count")).show(1,false)

+--------------------------+------+
|resource                  |count |
+--------------------------+------+
|/images/NASA-logosmall.gif|208353|
+--------------------------+------+
only showing top 1 row



### ¿Qué días la web recibió más tráfico?

In [19]:
val dateDF=parqDF.withColumn("date",substring_index(col("date"),":",1))
dateDF.groupBy(col("date")).agg(count("date")).orderBy(desc("count(date)")).show(5,false)

+-----------+-----------+
|date       |count(date)|
+-----------+-----------+
|13/Jul/1995|133883     |
|06/Jul/1995|100833     |
|05/Jul/1995|94445      |
|12/Jul/1995|92094      |
|31/Aug/1995|89739      |
+-----------+-----------+
only showing top 5 rows



dateDF: org.apache.spark.sql.DataFrame = [Host: string, date: string ... 5 more fields]


### ¿Cuáles son los hosts son los más frecuentes?

In [15]:
parqDF.groupBy(col("Host")).agg(count("Host")).orderBy(desc("count(Host)")).show(5,false)

+--------------------+-----------+
|Host                |count(Host)|
+--------------------+-----------+
|piweba3y.prodigy.com|21988      |
|piweba4y.prodigy.com|16437      |
|piweba1y.prodigy.com|12825      |
|edams.ksc.nasa.gov  |11962      |
|163.206.89.4        |9697       |
+--------------------+-----------+
only showing top 5 rows



### ¿A qué horas se produce el mayor número de tráfico en la web?

In [23]:
val horaDF=parqDF.withColumn("hora",substring(col("date"),13,2))
horaDF.groupBy(col("hora")).agg(count("hora")).orderBy(desc("count(hora)")).show(5,false)

+----+-----------+
|hora|count(hora)|
+----+-----------+
|15  |230298     |
|12  |226928     |
|13  |225082     |
|14  |223396     |
|16  |217143     |
+----+-----------+
only showing top 5 rows



horaDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 6 more fields]


### ¿Cuál es el número de errores 404 que ha habido cada día?

In [24]:
dateDF.printSchema()

root
 |-- Host: string (nullable = true)
 |-- date: string (nullable = true)
 |-- RequestMethod: string (nullable = true)
 |-- Resource: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- HTTPstatuscode: string (nullable = true)
 |-- Size: string (nullable = true)



In [29]:
dateDF.where(col("HTTPstatuscode")==="404").groupBy("date").agg(count("*").alias("404s")).orderBy(desc("404s")).show()

+-----------+----+
|       date|404s|
+-----------+----+
|19/Jul/1995| 636|
|06/Jul/1995| 630|
|30/Aug/1995| 564|
|07/Jul/1995| 563|
|31/Aug/1995| 525|
|13/Jul/1995| 524|
|07/Aug/1995| 523|
|05/Jul/1995| 492|
|03/Jul/1995| 473|
|11/Jul/1995| 469|
|18/Jul/1995| 463|
|12/Jul/1995| 459|
|25/Jul/1995| 458|
|20/Jul/1995| 427|
|24/Aug/1995| 419|
|25/Aug/1995| 411|
|29/Aug/1995| 411|
|14/Jul/1995| 407|
|28/Aug/1995| 405|
|17/Jul/1995| 403|
+-----------+----+
only showing top 20 rows

